In [2]:
from csw_search import *

In [3]:
from datetime import datetime

In [4]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, GeoJSON
from ipydatetime import DatetimePicker
from IPython.core.display import display
import ipyleaflet
from ipywidgets import widgets, Layout
from datetime import datetime, timezone
import dateutil
import dateutil.parser
import pytz
import warnings
warnings.filterwarnings('ignore')

# Clear output panel
def mk_clear_button(target, action_name):
    button = widgets.Button(description=action_name.replace('_', ' '))
    action = getattr(target, action_name)
    button.on_click(lambda *a: action())
    return button

# Launch search
def on_search_clicked(b):
    with out:
        out.clear_output()
        start = dateutil.parser.parse(str(datetime_picker_start.value))
        end = dateutil.parser.parse(str(datetime_picker_end.value))
        print("QUERY\nbbox=" + str(bbox) + "\nbetween " + str(start) + " and " + str(end)+ "\n")
        #records = csw_query(endpoint=csw_endpoints['IAGOS'], bbox=bbox, start=start, stop=end, kw_names=None, crs="urn:ogc:def:crs:OGC:1.3:CRS84")
        
def mk_search_button():
    button = widgets.Button(description="Search")
    button.on_click(on_search_clicked)
    return button

# Display coordinates
def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousemove':
        coords = kwargs.get('coordinates')
        lat_label.value = str(coords[0]) 
        lon_label.value = str(coords[1])

def handle_click(**kwargs):
    with out:
        print(kwargs)
        print(type(kwargs))
        # kwargs['feature']['properties'][foobar] = 10 
        
def on_draw_handler(draw, action, geo_json):
    with out:
        out.clear_output()
        for i in map_panel.layers:
            if type(i) == ipyleaflet.GeoJSON:
                map_panel.remove_layer(i)
        bounds = geo_json['geometry']['coordinates'][0]
        ll = bounds[0]
        ur = bounds[2]
        corners = [ll, ur]
        global bbox
        bbox = [item for sublist in corners for item in sublist]
        searchbox = GeoJSON(data=geo_json)
        searchbox.on_click(handle_click)
        map_panel.add_layer(searchbox)
        draw.clear_rectangles()

def selection_variables_list(values):
    return widgets.SelectMultiple(options=values,
    rows=10,
    description='Variables',
    disabled=False,
    layout=Layout(width='800px',)
)
        
def get_start_date():
    return datetime(2000, 1, 1, 0, 0, tzinfo=pytz.utc)

def get_end_date():
    return datetime.now(timezone.utc)
    
csw_endpoints = {'SIOS' : 'https://sios.csw.met.no', 'IAGOS' : 'http://catalogue2.sedoo.fr/geonetwork/srv/eng/csw-iagos' }
rest_endpoints = { 'ACTRIS' : 'https://prod-actris-md.nilu.no' }
accepted_vocabularies = { "Climate and Forecast", "GCMD"}

# Temporal extent
datetime_picker_start = DatetimePicker(value=get_start_date())
datetime_picker_end = DatetimePicker(value=get_end_date())

# Output panel
out = widgets.Output(layout=widgets.Layout(width='50%', 
                               height='400px', 
                               overflow_y='scroll',
                               border='1px solid black'))
clear_output = mk_clear_button(out, 'clear_output')
search = mk_search_button()

# Map panel
map_panel = ipyleaflet.Map(layers=(basemap_to_tiles(basemaps.Stamen.Terrain), ), center=(49, 11), zoom=4)
controls = ipyleaflet.LayersControl(position='topright')
lat_label = widgets.Label()
lon_label = widgets.Label()
map_panel.add_control(controls)
map_panel.on_interaction(handle_interaction)
global bbox
bbox = [-180, -90, 180, 90]

list_variables = selection_variables_list(get_csw_keywords(csw_endpoints, accepted_vocabularies))

#Draw panel for selection on map
draw = ipyleaflet.DrawControl(
    edit=True,
    remove=True,
    circlemarker={},
    marker={},
    circle={},    
    polyline={},
    polygon = {},
    rectangle={'shapeOptions': {}})
draw.on_draw(on_draw_handler)
map_panel.add_control(draw)

dashboard = widgets.VBox([ 
                    widgets.HBox([datetime_picker_start, datetime_picker_end]), 
                    widgets.HBox([list_variables]), 
                    widgets.HBox([clear_output, search]),
                    widgets.HBox([map_panel, out]), widgets.HBox([lon_label, lat_label])])

dashboard


ParseError: not well-formed (invalid token): line 1, column 0 (<string>)